In [2]:
import tskit
import pandas
import numpy

from tqdm import tqdm
pandas.options.display.max_columns=999
import tsconvert

import warnings
warnings.filterwarnings('ignore')

Read in the Newick file downloaded from ITOL. To get this I uploaded the Newick file in the `Brankin_Malone` repo and then exported it again as a Newick. Don't know why but trying to use the original Newick leads to errors in `tsconvert` lower down

In [3]:
with open('cryptic_tree.itol.newick', 'r') as f:
    cryptic_newick = f.read()

print(cryptic_newick[:1000])

(site.04.iso.1.subject.03818.lab_id.830476.seq_reps.1:6.90800,site.04.iso.1.subject.00310.lab_id.701951.seq_reps.1:3.09200,(site.04.iso.1.subject.01627.lab_id.27972.seq_reps.1:0.46454,(site.04.iso.1.subject.01042.lab_id.717652.seq_reps.1:8.18534,(((site.04.iso.1.subject.02112.lab_id.803805.seq_reps.1:9.91015,site.04.iso.1.subject.04137.lab_id.832790.seq_reps.1:8.08985):0.17857,((((site.04.iso.1.subject.01432.lab_id.724551.seq_reps.1:1.73206,site.04.iso.1.subject.02235.lab_id.805177.seq_reps.1:4.26794):0.87800,(site.04.iso.1.subject.00300.lab_id.702487.seq_reps.1:0.00000,site.04.iso.1.subject.03648.lab_id.JJH9682.seq_reps.1:2.00000):3.62200):8.94817,(site.04.iso.1.subject.00564.lab_id.709046.seq_reps.1:7.96229,(site.04.iso.1.subject.04356.lab_id.903933.seq_reps.1:5.42753,(site.04.iso.1.subject.01175.lab_id.721997.seq_reps.1:1.00000,site.04.iso.1.subject.01288.lab_id.801278.seq_reps.1:0.00000):3.07247):2.53771):1.48933):0.11168,(((site.04.iso.1.subject.02152.lab_id.803055.seq_reps.1:0.89

Use `tsconvert` to convert to a succint tree sequence

In [4]:
ts = tsconvert.from_newick(cryptic_newick, min_edge_length=0.001)
print('done')

done


Get a copy of the underlying tables that describe the tree so we can relate the `node id` to the `UNIQUEID`

In [5]:
new_tables = ts.dump_tables()  # make a copy of the tree sequence tables, for parsing
new_tables.nodes[:3]

id,flags,population,individual,time,metadata
0,0,-1,-1,2060.10904000,{}
1,1,-1,-1,2053.20104000,{'name': 'site.04.iso.1.subject.03818...
2,1,-1,-1,2057.01704000,{'name': 'site.04.iso.1.subject.00310...


Read in necessary data

In [11]:
#load mutations table for RNAP
mutations_agg = pandas.read_csv('/Users/viktoriabrunner/Documents/Studium/PhD/DPhil/rnap_minors/tb-rnap-subpopulations/data/mutations_agg-UNIQUEID.csv')

GENOMES = pandas.read_pickle('/Users/viktoriabrunner/Documents/Studium/PhD/DPhil/paper/tb-rnap-compensation/tb_rnap_compensation/tables/GENOMES.pkl.gz')
GENOMES.reset_index(inplace=True)

#add FTP_FILENAME_VCF from GENOMES table to mutations_agg table based on UNIQUEID
mutations_agg['FTP_FILENAME_VCF'] = mutations_agg['UNIQUEID'].map(GENOMES.set_index('UNIQUEID')['FTP_FILENAME_VCF'])
# mutations_agg.reset_index(inplace=True)
mutations_agg

def create_original_uid(row):

    filename = row.FTP_FILENAME_VCF

    if pandas.isna(filename):
        return None

    if '.regeno' in filename:
        if filename[:5] != 'comas':
            return('site'+filename.split('/site')[1].split('.regeno')[0])
        else:
            return(filename.split('comas_regeno/')[1].split('.regeno')[0])
    else:
        return None

mutations_agg['ORIGINAL_UID'] = mutations_agg.apply(create_original_uid, axis=1)

mutations_agg.set_index('ORIGINAL_UID', inplace=True)
mutations_agg.drop('FTP_FILENAME_VCF', axis=1, inplace=True)
mutations_agg

,UNIQUEID,RESISTANT_MUTATION,COMPENSATORY_MUTATION,R_MINOR_ALLELE,R_MAJOR_ALLELE,FRS
ORIGINAL_UID,,,,,,
None,site.00.subj.1000347.lab.H111540004.iso.1,True,False,False,True,NaN
None,site.00.subj.1000595.lab.H123460044.iso.1,True,True,False,True,NaN
None,site.00.subj.1004213.lab.H111060034.iso.1,True,True,False,True,NaN
None,site.00.subj.1004213.lab.H112000008.iso.1,True,True,False,True,NaN
None,site.00.subj.1004213.lab.H113100007.iso.1,True,True,False,True,NaN
...,...,...,...,...,...,...
None,site.35.subj.95.lab.IE19.iso.1,True,True,False,True,NaN
None,site.35.subj.96.lab.IE20.iso.1,True,False,False,True,NaN
None,site.35.subj.97.lab.IE21.iso.1,True,False,False,True,NaN


In [7]:
#create script to mark major and minor RIF resistant samples in tree
vis = 'binary'

binary_header="""DATASET_BINARY

SEPARATOR COMMA

DATASET_LABEL,binary

COLOR,#e41a1c

FIELD_SHAPES,1

FIELD_LABELS,f1

DATA
"""

binary_annotations = ''

for i in tqdm(range(ts.num_nodes)):   

        row = new_tables.nodes[i]

        if 'name' in row.metadata.keys():

            uid = row.metadata['name']

            if uid in mutations_agg[mutations_agg['R_MINOR_ALLELE']].index:
                line = 'n'+str(i) +',1\n'
                binary_annotations += line 

binary_annotations = binary_header + binary_annotations
    
title = 'minor_res-annotations_circle.txt'
print(title)

with open(title,'w') as f:
        f.write(binary_annotations)

 73%|███████▎  | 22264/30454 [00:04<00:01, 4859.03it/s]


KeyboardInterrupt: 

In [12]:
vis = 'symbol'

symbol_header="""DATASET_SYMBOL

SEPARATOR COMMA

DATASET_LABEL,example symbols

COLOR,#e41a1c

MAXIMUM_SIZE,3


DATA
"""

symbol_annotations = ''

for i in tqdm(range(ts.num_nodes)):   

        row = new_tables.nodes[i]

        if 'name' in row.metadata.keys():

            uid = row.metadata['name']

            if uid in mutations_agg[mutations_agg['R_MINOR_ALLELE']].index:
                line = 'n'+str(i) +',1,1,' + '#e41a1c' + ',1,1\n' ##e41a1c, #4daf4a
                symbol_annotations += line 
    

symbol_annotations = symbol_header + symbol_annotations

title = 'minor_res-annotations_symbol_red.txt'
print(title)

with open(title,'w') as f:
        f.write(symbol_annotations)

100%|██████████| 30454/30454 [00:08<00:00, 3456.06it/s]

minor_res-annotations_symbol_red.txt
